In [2]:
# Import the OpenCV library for computer vision and image processing.
import cv2 as cv # conda install -c conda-forge opencv
# Import the Matplotlib library for data visualization and image display.
import matplotlib.pyplot as plt # conda install matplotlib

# Load a neural network model from a TensorFlow-based model file. 
net = cv.dnn.readNetFromTensorflow('data/graph_opt.pb') ## weights

# Define input dimensions for the neural network.
inWidth = 368
inHeight = 368
# Set a threshold for body part detection confidence.
thr = 0.2

# Define a mapping of body part names to their corresponding indices.
BODY_PARTS = {
    "Nose": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3,
    "RWrist": 4, "LShoulder": 5, "LElbow": 6,
    "LWrist": 7, "RHip": 8, "RKnee": 9,
    "RAnkle": 10, "LHip": 11, "LKnee": 12,
    "LAnkle": 13, "REye": 14,
    "LEye": 15, "REar": 16, "LEar": 17,
    "Background": 18
}

# Define pairs of body parts that represent connections for visualization.
POSE_PAIRS = [
    ["Neck", "RShoulder"], ["Neck", "LShoulder"], ["RShoulder", "RElbow"],
    ["RElbow", "RWrist"], ["LShoulder", "LElbow"], ["LElbow", "LWrist"],
    ["Neck", "RHip"], ["RHip", "RKnee"], ["RKnee", "RAnkle"],
    ["Neck", "LHip"], ["LHip", "LKnee"], ["LKnee", "LAnkle"],
    ["Neck", "Nose"], ["Nose", "REye"],
    ["REye", "REar"], ["Nose", "LEye"], ["LEye", "LEar"]
]


In [3]:
# Open a video file named 'baby.mp4' for reading.
cap = cv.VideoCapture('baby.mp4')

# Set the width and height of the video frame to 800 pixels.
cap.set(3, 800)
cap.set(4, 800)

# Check if the video capture is successfully opened. If not, try to use the default camera (camera index 1).
if not cap.isOpened():
    cap = cv.VideoCapture(1)

# If the video capture is still not opened, raise an error.
if not cap.isOpened():
    raise IOError("Cannot open video")

# Start a loop for processing video frames.
while cv.waitKey(1) < 0:
    # Read the next frame from the video.
    hasFrame, frame = cap.read()

    # If there are no more frames, exit the loop.
    if not hasFrame:
        cv.waitKey()
        break

    # Get the width and height of the frame.
    frameWidth = frame.shape[1]
    frameHeight = frame.shape[0]

    # Prepare the frame for input to the neural network and pass it through the network.
    net.setInput(cv.dnn.blobFromImage(frame, 1.0, (inWidth, inHeight), (127.5, 127.5, 127.5), swapRB=True, crop=False))
    out = net.forward()

    # Extract the relevant part of the network output (only 19 elements are needed).
    out = out[:, :19, :, :]

    # Ensure the number of body parts defined is the same as the number of output elements.
    assert(len(BODY_PARTS) == out.shape[1])

    # Create an empty list to store points (coordinates of detected body parts).
    points = []

    for i in range(len(BODY_PARTS)):
        # Slice the heatmap corresponding to the body part.
        heatMap = out[0, 1, :, :]

        # Find the maximum confidence point in the heatmap.
        _, conf, _, point = cv.minMaxLoc(heatMap)
        x = (frameWidth * point[0]) / out.shape[3]
        y = (frameHeight * point[1]) / out.shape[2]

        # Add a point to the list if its confidence is higher than the threshold.
        points.append((int(x), int(y)) if conf > thr else None)

    for pair in POSE_PAIRS:
        partFrom = pair[0]
        partTo = pair[1]
        assert(partFrom in BODY_PARTS)
        assert(partTo in BODY_PARTS)

        idFrom = BODY_PARTS[partFrom]
        idTo = BODY_PARTS[partTo]

        if points[idFrom] and points[idTo]:
            cv.line(frame, points[idFrom], points[idTo], (0, 255, 0), 3)
            cv.ellipse(frame, points[idFrom], (3, 3), 8, 8, 368, (0, 0, 255), cv.FILLED)
            cv.ellipse(frame, points[idTo], (3, 3), 8, 8, 360, (0, 0, 255), cv.FILLED)

    t, _ = net.getPerfProfile()
    freq = cv.getTickFrequency() / 1000
    cv.putText(frame, '%.2fms' % (t / freq), (18, 28), cv.FONT_HERSHEY_SIMPLEX, 8.5, (0, 0, 0))

    # Display the resulting video frame with estimated pose.
    cv.imshow('Pose estimation tutorial', frame)


KeyboardInterrupt: 